In [12]:
import pickle

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torch.nn import Embedding, RNN
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

# Set any global values

In [13]:
torch.manual_seed(1)

In [14]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


# Load the training data

In [15]:
with open('training_data.pickle','rb') as f:
    training_data = pickle.load(f)

In [16]:
class_to_ix = {"Found":0, "Unfound":1}
all_letters = "()ab~&|>"

vocab = ['<pad>'] + sorted(set([char for seq in all_letters for char in seq]))


n_classes = len(class_to_ix)
n_letters = len(all_letters)

In [17]:
X = []
Y = []
embed_dim = len(vocab)
embed = Embedding(len(vocab), embed_dim) # embedding_dim = len(vocab)
for element in training_data:
    input = [vocab.index(token) for token in element[0]]
    input_tensor = torch.tensor(input, dtype=torch.int)
    classification = class_to_ix[element[1]]
    X.append(input_tensor)
    output_tensor = torch.tensor(classification, dtype=torch.int)
    Y.append(output_tensor)

In [18]:
class TTPStyleDataset(Dataset):
    def __init__(self, X,Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample = [self.X[idx], self.Y[idx]]
        return sample

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

batch_size = 16
input_dataset = TTPStyleDataset(X_train,y_train)

test_dataset = TTPStyleDataset(X_test,y_test)

def my_collate_fn(data):
    (xx,yy) = zip(*data)
    x_lens = [len(x) for x in xx]
    y_lens = [1 for y in yy]

    xx_pad = pad_sequence(xx, batch_first=False, padding_value=0)
    yy_pad = torch.tensor(yy)

    return xx_pad, yy_pad, x_lens, y_lens
    

dataloader = DataLoader(input_dataset, shuffle=True, batch_size=batch_size, num_workers=0, collate_fn=my_collate_fn)
test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size, num_workers=0, collate_fn=my_collate_fn)

# Neural Network

In [20]:
class SoftmaxRNN(nn.RNN):
    def __init__(self, input_size, hidden_size, num_layers, nonlinearity, bias, batch_first, dropout, bidirectional, output_size):
        super(SoftmaxRNN, self).__init__(input_size, hidden_size, num_layers, nonlinearity, bias, batch_first, dropout, bidirectional)
        self.fc = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, x, h_0=None):
        output, h = super(SoftmaxRNN, self).forward(x, h_0)
        output, seq_len = pad_packed_sequence(output, batch_first=False)
        output = self.fc(h[-1])
        return output, h

In [21]:
#input_size - The number of expected features in the input x
rnn = SoftmaxRNN(input_size=embed_dim, hidden_size=128, num_layers=1, nonlinearity='relu', bias=True, batch_first=False, dropout=0.0, bidirectional=False, output_size=2)
rnn.to(device)

SoftmaxRNN(
  9, 128
  (fc): Linear(in_features=128, out_features=2, bias=True)
)

In [ ]:
num_epochs = 10
learning_rate = 0.01 # If you set this too high, it might explode. If too low, it might not learn
optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate)

criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    print("epoch: "+str(epoch))
    size = len(dataloader.dataset)
    rnn.train()
    for batch,(x_padded, y_padded, x_lens, y_lens) in enumerate(dataloader):

        
        rnn.zero_grad()
        x_embed = embed(x_padded)
        x_packed = pack_padded_sequence(x_embed, x_lens, batch_first=False, enforce_sorted=False)
        x_packed = x_packed.to(device)
        y_padded = y_padded.to(device)
        output, hidden = rnn(x_packed)
        output = torch.reshape(output, (batch_size,2))
        
        y_padded = torch.reshape(y_padded, (batch_size,))
        y_padded = y_padded.long()
        loss = criterion(output, y_padded)

        loss.backward()
        optimizer.step()
        

        if batch % 500 == 0:
            x_s = np.shape(x_padded)[1]
            loss, current = loss.item(), (batch + 1) * x_s
            print(f"train loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")



    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    rnn.eval()
    size = len(test_dataloader.dataset)
    num_batches = len(test_dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for batch,(x_padded, y_padded, x_lens, y_lens) in enumerate(test_dataloader):
            rnn.zero_grad()
            x_embed = embed(x_padded)
            x_packed = pack_padded_sequence(x_embed, x_lens, batch_first=False, enforce_sorted=False)
            x_packed = x_packed.to(device)
            y_padded = y_padded.to(device)
            output, hidden = rnn(x_packed)
            output = torch.reshape(output, (batch_size,2))
            
            y_padded = torch.reshape(y_padded, (batch_size,))
            y_padded = y_padded.long()
            loss += criterion(output, y_padded)
        
        #loss, current = loss, (batch + 1) * len(x_padded)
        test_loss = loss/num_batches
        print(f"test loss: {test_loss:>7f}")

epoch: 0
train loss: 0.716086  [   16/141120]
train loss: 0.677070  [ 8016/141120]
train loss: 0.678187  [16016/141120]
train loss: 0.610904  [24016/141120]
train loss: 0.616658  [32016/141120]
train loss: 0.575241  [40016/141120]
train loss: 0.563723  [48016/141120]
train loss: 0.612464  [56016/141120]
train loss: 0.475252  [64016/141120]
train loss: 0.428484  [72016/141120]
train loss: 0.518136  [80016/141120]
train loss: 0.580194  [88016/141120]
train loss: 0.751507  [96016/141120]
train loss: 0.543724  [104016/141120]
train loss: 0.616102  [112016/141120]
train loss: 0.442388  [120016/141120]
train loss: 0.484365  [128016/141120]
train loss: 0.570755  [136016/141120]
test loss: 0.576827
epoch: 1
train loss: 0.753969  [   16/141120]
train loss: 0.483880  [ 8016/141120]
train loss: 0.843346  [16016/141120]
train loss: 0.523748  [24016/141120]
train loss: 0.655251  [32016/141120]
train loss: 0.441048  [40016/141120]
train loss: 0.561925  [48016/141120]
train loss: 0.551114  [56016/141